In [1]:
import time
import psycopg2
import argparse
import re
import csv
import pandas as pd
import json
from datetime import datetime
import os
import ast
from data_transform import *
import time
import psycopg2.extras as extras

In [30]:
def return_df():
    # current_file_path = "/home/reeya/consumed_data"
    # date = datetime.today().strftime('%Y-%m-%d')
    # file_name = date + str(".json")
    # file_path = os.path.join(current_file_path, file_name)

    file_path = "/home/reeya/consumed_stop_event/2022-05-25.json"

    with open(file_path, 'r') as f:
        data = json.load(f)

    print("Data Loaded..")

        # print(data[0])
    for i in range(len(data)):
        data_string = ast.literal_eval(data[i])
        data[i] = json.loads(data_string)

    print("Data Transformed..")


    df = pd.read_json(json.dumps(data))
    # print(df.head(10))
    print("Returning Data")
    return df

In [31]:
df = return_df()

Data Loaded..
Data Transformed..
Returning Data


In [35]:
print(list(df.columns))

['vehicle_number', 'leave_time', 'train', 'route_number', 'direction', 'service_key', 'stop_time', 'arrive_time', 'dwell', 'location_id', 'door', 'lift', 'ons', 'offs', 'estimated_load', 'maximum_speed', 'train_mileage', 'pattern_distance', 'location_distance', 'x_coordinate', 'y_coordinate', 'data_source', 'schedule_status', 'trip_id']


In [38]:
np.where(df[['service_key']].applymap(lambda x: x == ''))

(array([], dtype=int64), array([], dtype=int64))

In [51]:
print("x_coordinate: ",np.where(pd.isnull(df['x_coordinate'])))
print("y_coordinate: ",np.where(pd.isnull(df['y_coordinate'])))
print("location_distance ",np.where(pd.isnull(df['location_distance'])))



x_coordinate:  (array([   30, 21090, 21183, 25364, 25427, 26432]),)
y_coordinate:  (array([   30, 21090, 21183, 25364, 25427, 26432]),)
location_distance  (array([   30, 21090, 21183, 25364, 25427, 26432]),)


In [52]:
print(list(df.columns))

['vehicle_number', 'leave_time', 'train', 'route_number', 'direction', 'service_key', 'stop_time', 'arrive_time', 'dwell', 'location_id', 'door', 'lift', 'ons', 'offs', 'estimated_load', 'maximum_speed', 'train_mileage', 'pattern_distance', 'location_distance', 'x_coordinate', 'y_coordinate', 'data_source', 'schedule_status', 'trip_id']


In [55]:
print(list(df.columns))

['vehicle_number', 'leave_time', 'train', 'route_number', 'direction', 'service_key', 'stop_time', 'arrive_time', 'dwell', 'location_id', 'door', 'lift', 'ons', 'offs', 'estimated_load', 'maximum_speed', 'train_mileage', 'pattern_distance', 'location_distance', 'x_coordinate', 'y_coordinate', 'data_source', 'schedule_status', 'trip_id']


In [80]:
column_names = ['route_number', 'direction', 'stop_time', 'arrive_time', 'vehicle_number', 'leave_time', 'train', 'dwell', 'location_id', 'door', 'lift', 'ons', 'offs','estimated_load', 'maximum_speed', 'data_source', 'schedule_status', 'trip_id']

for each_column_name in column_names:
    if(df[each_column_name].isnull().values.any()):
        df[each_column_name] = df[each_column_name].interpolate(method='linear', limit_direction='both').astype(int)

df = df.assign(maximum_speed = 0)


In [81]:
print("location_id: ",np.where(pd.isnull(df['location_id'])))
print("maximum_speed: ",np.where(pd.isnull(df['maximum_speed'])))

location_id:  (array([], dtype=int64),)
maximum_speed:  (array([], dtype=int64),)


In [76]:
DB_Name = "ctran_db"
DB_User = "reeya"
DB_Password = "reeya"
Create_DB = False
TableName = "teststopdata"
def db_connect():
    connection = psycopg2.connect(
        host = "localhost",
        database = DB_Name,
        user = DB_User,
        password = DB_Password,
    )
    connection.autocommit = True
    return connection

In [77]:
def execute_batch_trip(conn, df, table, page_size=100):
    """
    Using psycopg2.extras.execute_batch() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s)" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_batch(cursor, query, tuples, page_size)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_batch() done")
    cursor.close()


In [78]:
conn = db_connect()

In [84]:
execute_batch_trip(conn=conn, df=df, table='teststopdata', page_size=100)

execute_batch() done


In [82]:
print(df[:1])

   vehicle_number  leave_time  train  route_number  direction service_key  \
0            4001       19683   7101            71        1.0           W   

   stop_time  arrive_time  dwell  location_id  ...  estimated_load  \
0    20160.0        19683      0        543.0  ...               3   

   maximum_speed  train_mileage  pattern_distance  location_distance  \
0              0         1.4838            1.4882           13.42337   

   x_coordinate  y_coordinate  data_source  schedule_status    trip_id  
0   -122.661923      45.67998            2                2  171386740  

[1 rows x 24 columns]
